In [ ]:
# Import Libraries =============================================================
import os
import time
import shutil
import subprocess
try:
    from google.colab import drive  # type:ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Set Configurations ===========================================================
BASE_PATH = '/content'
MOUNT_PATH = '/content/drive/MyDrive'
SRC_PATH = os.path.join(MOUNT_PATH, 'src')
REPO_PATH = os.path.join(SRC_PATH, 'ml-playverse')
MAIN_PATH = ['cifar']
FORCE_MOUNT = False
FORCE_CLONE = False
PROCESS_DELAY = 5
PROCESS_TIMEOUT = 25

# Mount Drive ===============================================================
print("\n🔗 Mounting Google Drive on Colab")
if IN_COLAB:
    if FORCE_MOUNT:
        drive.mount("/content/drive", force_remount=True)
    else:
        drive.mount("/content/drive")
print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish mounting")
time.sleep(PROCESS_DELAY)
for _ in range(PROCESS_TIMEOUT):
    if os.path.exists(MOUNT_PATH):
        break
    time.sleep(1)
else:
    raise TimeoutError("❌ Google Drive mounting timed out")
print("\n✅ Google Drive has been mounted on Colab")

# if FORCE_CLONE: Delete Source =============================================
if os.path.exists(SRC_PATH):
    if FORCE_CLONE:
        print("\n🗑️ Deleting source directory")
        shutil.rmtree(SRC_PATH)
        os.chdir(BASE_PATH)
        print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish deleting")
        time.sleep(PROCESS_DELAY)
        for _ in range(PROCESS_TIMEOUT):
            if not os.path.exists(SRC_PATH):
                break
            time.sleep(1)
        else:
            raise TimeoutError("❌ Source directory deletion timed out")
        print("\n✅ Source directory has been deleted")
    else:
        print("\n⚠️ Source directory has been preserved")
else:
    print("\n✅ Source directory was already deleted")

# if FORCE_CLONE: Create Source =============================================
if FORCE_CLONE:
    print("\n📁 Creating source directory")
    os.makedirs(SRC_PATH, exist_ok=True)
    print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish creating")
    time.sleep(PROCESS_DELAY)
    for _ in range(PROCESS_TIMEOUT):
        if os.path.exists(SRC_PATH):
            break
        time.sleep(1)
    else:
        raise TimeoutError("❌ Source directory creation timed out")
    print("\n✅ Source directory has been created")
else:
    print("\n⚠️ Source directory was preserved")

# Prepare Repository ========================================================
print("\n📥 Preparing repository on Colab")

if not FORCE_CLONE and os.path.exists(REPO_PATH):
    print("\n🔄 Pulling repository from GitHub")
    modification_time = os.path.getmtime(os.path.join(REPO_PATH, '.git'))

    # Prefer remote state over local
    !git -C "{REPO_PATH}" fetch origin
    !git -C "{REPO_PATH}" reset --hard origin/main

    print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish pulling")
    time.sleep(PROCESS_DELAY)
    for _ in range(PROCESS_TIMEOUT):
        current_mtime = os.path.getmtime(os.path.join(REPO_PATH, '.git'))
        if current_mtime != modification_time:
            break
        time.sleep(1)
    else:
        raise TimeoutError("❌ Repository pulling timed out")
    print("\n✅ Repository has been pulled")
else:
    print("\n🌱 Cloning repository from GitHub")
    !git clone https://github.com/saeedkhademidoroh/ml-playverse.git "{REPO_PATH}"
    print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish cloning")
    time.sleep(PROCESS_DELAY)
    for _ in range(PROCESS_TIMEOUT):
        if os.path.isdir(REPO_PATH) and os.listdir(REPO_PATH):
            break
        time.sleep(1)
    else:
        raise TimeoutError("❌ Repository cloning timed out")
    print("\n✅ Repository has been cloned")


# Run Main ==================================================================
for main in MAIN_PATH:
    main_path = os.path.join(REPO_PATH, main)
    print(f"\n📂 Changing directory:\n{main_path}")
    if os.path.exists(main_path):
        os.chdir(main_path)
        print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish changing")
        time.sleep(PROCESS_DELAY)
        print("\n✅ Directory has been changed")
        print("\n🚀 Running main.py as a subprocess")
        process = subprocess.Popen(
            ["python", "-u", "main.py"],
            cwd=main_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        for line in process.stdout:
            print(line, end='')
        process.wait()
    else:
        raise FileNotFoundError(f"❌ Changing directory timed out")